<a href="https://colab.research.google.com/github/akhanna99/transformer/blob/main/transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import collections
import numpy as np
# hyperparameters
batch_size = 8 #32 how many independent sequences will we process in parallel?
block_size = 512#512#512#12 #16 what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 50
learning_rate = 1e-3 #3e-4 #6e-4
'''
weight_decay = 1e-1
beta1 = 0.9
betas2 = 0.95
grad_clip = 1.0
decay_lr =True
10M parameters 300k tokens
'''


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 768
n_head = 8
n_layer = 8
dropout = 0.0
# ------------
initialTrain=True
retrain=False

cpu


In [ ]:
# read in text from google drive
import os
from google.colab import drive
import sys

drive.mount('/content/MyDrive')

path = '/content/MyDrive/MyDrive/Colab Notebooks/alice_in_wonderland.txt'

with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
# tokenisation
'''
# here are all the unique characters that occur in this text
# IMPORTANT:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
'''

"\n# here are all the unique characters that occur in this text\n# IMPORTANT: \nchars = sorted(list(set(text)))\nvocab_size = len(chars)\n# create a mapping from characters to integers\nstoi = { ch:i for i,ch in enumerate(chars) }\nitos = { i:ch for i,ch in enumerate(chars) }\nencode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers\ndecode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string\n"

In [ ]:

words = text.split()
word_counts = {word: count for word, count in collections.Counter(words).items()}
unique_words = sorted(word_counts.keys(), key=lambda word: word_counts[word], reverse=True)
vocab_size = len(unique_words)
stoi = {word: i for i, word in enumerate(unique_words)}
itos = {i: word for i, word in enumerate(unique_words)}
encode = lambda s: [stoi[word] for word in s.split()]
decode = lambda l: ' '.join([itos[i] for i in l])


In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
#data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    # Linear projections we are going to apply to all nodes

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention running in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # projection is a linear transformation of the outcome of the previous layer
        out = self.dropout(self.proj(out))
        return out # projection back into the residual pathway

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection layer going back into the residual pathway
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # 'fork off' and do some communication and then come back
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context

        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
def early_stopping(train_loss, val_loss, min_delta, tollerance):
    counter = 0
    if (val_loss - train_loss) > min_delta:
        counter+=1
        if counter >= tollerance:
            return True

In [ ]:
class EarlyStopper:
    def __init__(self,patience=1,min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter=0
        self.min_val_loss = np.inf

    def early_stop(self, val_loss, model):
        if val_loss < self.min_val_loss:
            self.min_val_loss = val_loss
            self.counter = 0
            # save the trained model
            model_path = "trained_model.pth"
            torch.save(model.state_dict(), model_path)
        elif val_loss > (self.min_val_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
            return False

In [ ]:
# Train model function

def train_model(model, optimizer, max_iters, eval_interval):
    early_stopper = EarlyStopper(patience = 5, min_delta = 0.01)
    for iter in range(max_iters):
        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        if early_stopper.early_stop(losses['train'],model):
            print('Early stopping')
            break


        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        model.train()
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()



In [ ]:
if initialTrain:
    # Initial train
    model = BigramLanguageModel()
    print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    model.train()
    train_model(model, optimizer, max_iters, eval_interval)
    # save the trained model
    #model_path = "trained_model.pth"
    #torch.save(model.state_dict(), model_path)

65.234618 M parameters
step 0: train loss 8.7275, val loss 8.7495
step 50: train loss 6.7363, val loss 7.7959
step 100: train loss 6.1798, val loss 7.4179
step 150: train loss 5.5249, val loss 7.3641
step 200: train loss 4.4481, val loss 7.4228
step 250: train loss 3.2232, val loss 7.5379
step 300: train loss 2.2092, val loss 7.9018
step 350: train loss 1.4979, val loss 8.2788
step 400: train loss 0.9800, val loss 8.7977
step 450: train loss 0.6216, val loss 9.2144
step 500: train loss 0.4161, val loss 9.5471
step 550: train loss 0.2979, val loss 9.7525
step 600: train loss 0.2336, val loss 9.9002
step 650: train loss 0.1846, val loss 10.0383
step 700: train loss 0.1445, val loss 10.2890
step 750: train loss 0.1253, val loss 10.4174
step 800: train loss 0.1094, val loss 10.5045
step 850: train loss 0.0940, val loss 10.5650
step 900: train loss 0.0822, val loss 10.6547
step 950: train loss 0.0781, val loss 10.6126
step 1000: train loss 0.0679, val loss 10.6945
step 1050: train loss 0.05

In [ ]:
if retrain:
    # Retrain the model
    model = BigramLanguageModel()
    model_path = "trained_model.pth"
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.train()
    learning_rate = 1e-2
    max_iters = 1500
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    train_model(model, optimizer, max_iters, eval_interval)
    print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
    # save the trained model
    model_path = "trained_model.pth"
    torch.save(model.state_dict(), model_path)

In [ ]:
# Inference code for generating new tokens

# Load the trained model
model = BigramLanguageModel()
model_path = "/content/MyDrive/MyDrive/Colab Notebooks/trained_model_joe.pth"
model.load_state_dict(torch.load(model_path, map_location='cpu'))
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
model = model.to(device)
model.eval()

# Generate from the loaded model

context = torch.zeros((1, 1), dtype=torch.long, device=device)

prompt = "Alice was beginning to"
context2 = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)

generated_tokens = model.generate(context2, max_new_tokens=300)[0].tolist()
generated_text = decode(generated_tokens)
#generated_text = decode(generated_tokens)
print(generated_text)

65.234618 M parameters
Alice was beginning to see its meaning. `And just as I'd taken the highest tree in the wood,' continued the Pigeon, raising its voice to a shriek, `and just as I was thinking I should be free of them at last, they must needs come wriggling down from the sky! Ugh, Serpent!' `But I'm NOT a serpent, I tell you!' said Alice. `I'm a--I'm a--' `Well! WHAT are you?' said the Pigeon. `I can see you're trying to invent something!' `I--I'm a little girl,' said Alice, rather doubtfully, as she remembered the number of changes she had gone through that day. `A likely story indeed!' said the Pigeon in a tone of the deepest contempt. `I've seen a good many little girls in my time, but never ONE with such a neck as that! No, no! You're a serpent; and there's no use denying it. I suppose you'll be telling me next that you never tasted an egg!' `I HAVE tasted eggs, certainly,' said Alice, who was a very truthful child; `but little girls eat eggs quite as much as serpents do, you 